# 第2回講義 宿題

## 課題
今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) をソフトマックス回帰によって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- 訓練データは`x_train`， `y_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください．**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### データの読み込み（このセルは修正しないでください）

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/y_train.npy')

    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255

    return x_train, y_train, x_test

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('x_train.npy')
    y_train = np.load('y_train.npy')

    # テストデータ
    x_test = np.load('x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255

    return x_train, y_train, x_test

### ソフトマックス回帰の実装

In [26]:
x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / x_exp.sum(axis=1, keepdims=True)

# 重み
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
b = np.zeros(shape=(10,)).astype('float32')

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, y, eps=1.0):
    global W, b

    batch_size = x.shape[0]

    y_hat = softmax(np.matmul(x, W) + b)
    cost = (-y * np_log(y_hat)).sum(axis=1).mean()
    delta = y_hat - y

    dW = np.matmul(x.T, delta) / batch_size
    db = np.matmul(np.ones(shape=(batch_size,)), delta)  / batch_size
    W -= eps * dW
    b -= eps * db
    return cost

def valid(x, y):
    y_hat = softmax(np.matmul(x, W) + b)
    cost = (-y * np_log(y_hat)).sum(axis=1).mean()
    return cost, y_hat

initial_eps = 1.0
eps = initial_eps

best_valid_cost = float('inf')
patience = 0
patience_threshold = 5
improvement_threshold = 1e-4
eps_reduction_factor = 0.8
eps_minimum_threshold = 0.15

for epoch in range(1000):
    cost = train(x_train, y_train, eps=eps)

    cost, y_pred = valid(x_valid, y_valid)

    if best_valid_cost - cost > improvement_threshold:
        best_valid_cost = cost
        patience = 0
    else:
        patience += 1
    
    if patience > patience_threshold and eps*eps_reduction_factor > eps_minimum_threshold:
        eps *= eps_reduction_factor
        patience = 0
        print('Learning rate reduced to {:.5f}'.format(eps))

    if epoch % 10 == 9 or epoch == 0:
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost,
            accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
        ))

y_pred = softmax(np.matmul(x_test, W) + b)

submission = pd.Series(y_pred.argmax(axis=1), name='label')
submission.to_csv('submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 2.908, Valid Accuracy: 0.343
Learning rate reduced to 0.80000
EPOCH: 10, Valid Cost: 6.476, Valid Accuracy: 0.376
Learning rate reduced to 0.64000
EPOCH: 20, Valid Cost: 2.825, Valid Accuracy: 0.637
Learning rate reduced to 0.51200
EPOCH: 30, Valid Cost: 1.088, Valid Accuracy: 0.724
Learning rate reduced to 0.40960
Learning rate reduced to 0.32768
EPOCH: 40, Valid Cost: 0.857, Valid Accuracy: 0.770
EPOCH: 50, Valid Cost: 0.859, Valid Accuracy: 0.769
EPOCH: 60, Valid Cost: 0.836, Valid Accuracy: 0.772
EPOCH: 70, Valid Cost: 0.823, Valid Accuracy: 0.777
EPOCH: 80, Valid Cost: 0.808, Valid Accuracy: 0.780
EPOCH: 90, Valid Cost: 0.802, Valid Accuracy: 0.783
EPOCH: 100, Valid Cost: 0.777, Valid Accuracy: 0.785
EPOCH: 110, Valid Cost: 0.817, Valid Accuracy: 0.786
EPOCH: 120, Valid Cost: 0.712, Valid Accuracy: 0.792
Learning rate reduced to 0.26214
Learning rate reduced to 0.20972
EPOCH: 130, Valid Cost: 0.598, Valid Accuracy: 0.811
EPOCH: 140, Valid Cost: 0.545, Valid A